In [3]:
import boto3
import os

SPACES_REGION = "us-east-1"
SPACES_ENDPOINT_URL = "https://apmc-ml-models.sfo3.digitaloceanspaces.com"
SPACES_ACCESS_KEY = "DO801T4Y2XGLAGTN2TDP"
SPACES_SECRET_KEY = "PtSEZ4g6Sdo6JY1AFQffrXAueiXghu+XMvxnjYMu0gM"
SPACES_BUCKET_NAME = "apmc-ml-models"

print(SPACES_REGION)
print(SPACES_ENDPOINT_URL)
print(SPACES_ACCESS_KEY)
print(SPACES_SECRET_KEY)
print(SPACES_BUCKET_NAME)

def get_spaces_client():
    return boto3.client(
        's3',
        region_name=SPACES_REGION,
        endpoint_url=SPACES_ENDPOINT_URL,
        aws_access_key_id=SPACES_ACCESS_KEY,
        aws_secret_access_key=SPACES_SECRET_KEY
    )

def list_categories(bucket_name, prefix):
    client = get_spaces_client()

    response = client.list_objects_v2(
        Bucket=bucket_name,
        Prefix=prefix,          # e.g., "ml_models/"
        Delimiter='/'           # <-- the magic part
    )

    categories = []
    if 'CommonPrefixes' in response:
        for cp in response['CommonPrefixes']:
            # Remove prefix base to get only folder name
            folder_name = cp['Prefix'][len(prefix):].rstrip('/')
            categories.append(folder_name)

    return categories

def test():
    client = get_spaces_client()
    print(client.list_buckets())
    print(client.list_objects(Bucket="apmc-ml-models"))

if __name__ == "__main__":
    print("🧾 Listing categories in 'ml_models/' ...")

    # categories = list_categories(SPACES_BUCKET_NAME, "ml_models/")
    # for cat in categories:
    #     print(f"📁 {cat}")
    
    test()


us-east-1
https://apmc-ml-models.sfo3.digitaloceanspaces.com
DO801T4Y2XGLAGTN2TDP
PtSEZ4g6Sdo6JY1AFQffrXAueiXghu+XMvxnjYMu0gM
apmc-ml-models
🧾 Listing categories in 'ml_models/' ...
{'ResponseMetadata': {'RequestId': 'tx00000d231db30c596d742-0067fa4954-6e288cdf-sfo3a', 'HostId': '', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-request-id': 'tx00000d231db30c596d742-0067fa4954-6e288cdf-sfo3a', 'content-type': 'application/xml', 'date': 'Sat, 12 Apr 2025 11:07:00 GMT', 'strict-transport-security': 'max-age=15552000; includeSubDomains; preload', 'transfer-encoding': 'chunked'}, 'RetryAttempts': 0}, 'Prefix': ''}


NoSuchKey: An error occurred (NoSuchKey) when calling the ListObjects operation: None

In [22]:
def list_categories(bucket_name, prefix):
    client = get_spaces_client()

    response = client.list_objects_v2(
        Bucket=bucket_name,
        Prefix=prefix,          # Full path prefix!
        Delimiter='/'           # Tells Spaces to only return folders under this
    )

    categories = []
    if 'CommonPrefixes' in response:
        for obj in response['CommonPrefixes']:
            category = obj['Prefix'].split('/')[-2]  # Get the folder name
            categories.append(category)

    return categories


if __name__ == "__main__":
    print("🧾 Listing categories in 'ml_models/' ...")
    categories = list_categories("apmc-ml-models", "apmc-ml-models/ml_models/")
    for cat in categories:
        print(f"📁 {cat}")


🧾 Listing categories in 'ml_models/' ...


NoSuchKey: An error occurred (NoSuchKey) when calling the ListObjectsV2 operation: None

In [23]:
def list_categories(bucket_name, prefix):
    client = get_spaces_client()

    response = client.list_objects_v2(
        Bucket=bucket_name,
        Prefix=prefix,          # Correct prefix for inside the bucket
        Delimiter='/'           # Tells Spaces to split by folder
    )

    categories = []
    if 'CommonPrefixes' in response:
        for obj in response['CommonPrefixes']:
            category = obj['Prefix'].split('/')[-2]  # Get the folder name
            categories.append(category)

    return categories


if __name__ == "__main__":
    print("🧾 Listing categories in 'ml_models/' ...")
    categories = list_categories(
        "apmc-ml-models",
        "ml_models/"    # <-- FIXED: Remove 'apmc-ml-models/' from the prefix!
    )
    for cat in categories:
        print(f"📁 {cat}")


🧾 Listing categories in 'ml_models/' ...


NoSuchKey: An error occurred (NoSuchKey) when calling the ListObjectsV2 operation: None

In [24]:
def debug_list_all(bucket_name):
    client = get_spaces_client()

    response = client.list_objects_v2(
        Bucket=bucket_name,
    )

    if 'Contents' not in response:
        print("⚠️ No objects found in this bucket!")
        return

    print(f"🗂 Found {len(response['Contents'])} objects:\n")
    for obj in response['Contents']:
        print(f"🔹 {obj['Key']}")

if __name__ == "__main__":
    debug_list_all("apmc-ml-models")


NoSuchKey: An error occurred (NoSuchKey) when calling the ListObjectsV2 operation: None

In [25]:
def list_everything(bucket_name):
    client = get_spaces_client()
    response = client.list_objects_v2(Bucket=bucket_name)

    if 'Contents' not in response:
        print("⚠️ No files found in this bucket!")
    else:
        print("📋 Files in bucket:")
        for obj in response['Contents']:
            print(f"🔹 {obj['Key']}")

if __name__ == "__main__":
    list_everything('apmc-ml-models')


NoSuchKey: An error occurred (NoSuchKey) when calling the ListObjectsV2 operation: None

<ListBucketResult xmlns="http://s3.amazonaws.com/doc/2006-03-01/">
<Name>apmc-ml-models</Name>
<Prefix/>
<MaxKeys>1000</MaxKeys>
<IsTruncated>false</IsTruncated>
<Contents>
<Key>apmc-ml-models/data/</Key>
<LastModified>2025-04-09T09:07:40.254Z</LastModified>
<ETag>"1441a7909c087dbbe7ce59881b9df8b9"</ETag>
<Size>15</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/data/commodities/</Key>
<LastModified>2025-04-09T10:53:06.576Z</LastModified>
<ETag>"1441a7909c087dbbe7ce59881b9df8b9"</ETag>
<Size>15</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/data/commodities/commodities_price_data-all_years.csv</Key>
<LastModified>2025-04-11T12:49:43.278Z</LastModified>
<ETag>"3f6989ed59120f29ecca9ed7364576c2"</ETag>
<Size>1389464</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/data/commodities/commodities_price_data.csv</Key>
<LastModified>2025-04-11T12:49:31.495Z</LastModified>
<ETag>"d15355138d2d8048daf3ab6178475813"</ETag>
<Size>132115</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/data/fruits/</Key>
<LastModified>2025-04-09T09:08:18.715Z</LastModified>
<ETag>"1441a7909c087dbbe7ce59881b9df8b9"</ETag>
<Size>15</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/data/fruits/fruits_price_data-all_years.csv</Key>
<LastModified>2025-04-12T06:56:01.009Z</LastModified>
<ETag>"b01f65efc4cd637d86cedaebd8352386"</ETag>
<Size>594932</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/data/fruits/fruits_price_data.csv</Key>
<LastModified>2025-04-12T06:55:49.580Z</LastModified>
<ETag>"106a936c2414e65e7a54617e04eb10a0"</ETag>
<Size>64519</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/data/vegetables/</Key>
<LastModified>2025-04-09T09:33:22.610Z</LastModified>
<ETag>"1441a7909c087dbbe7ce59881b9df8b9"</ETag>
<Size>15</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/data/vegetables/vegetables_price_data-all_years.csv</Key>
<LastModified>2025-04-11T12:45:07.008Z</LastModified>
<ETag>"cc1f7160f5fb974715ab167eccb5cb5f"</ETag>
<Size>1921836</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/data/vegetables/vegetables_price_data.csv</Key>
<LastModified>2025-04-11T12:44:48.806Z</LastModified>
<ETag>"fb0d13cb4af0586d9e84bba07cdc9431"</ETag>
<Size>183853</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/</Key>
<LastModified>2025-04-09T10:52:49.166Z</LastModified>
<ETag>"1441a7909c087dbbe7ce59881b9df8b9"</ETag>
<Size>15</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_0b6e645ac8664f4d0541742ab121ed52.pkl</Key>
<LastModified>2025-04-11T12:52:00.623Z</LastModified>
<ETag>"3aca4d690b328fb0236a0bd06d2e14f1"</ETag>
<Size>558255</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_0e3fdde6c1a0062af52b0d30296682b4.pkl</Key>
<LastModified>2025-04-11T12:50:33.700Z</LastModified>
<ETag>"1a4c1e802a8e92c130661bc2cd9e8e32"</ETag>
<Size>798006</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_0e59e7efcba2b8d4f15c49aea2a2949d.pkl</Key>
<LastModified>2025-04-11T12:52:36.135Z</LastModified>
<ETag>"55efcbbbcb920d9d854bf57d43d74c3a"</ETag>
<Size>334095</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_0ec1fbd98dc1bb0aa0c2344dad082577.pkl</Key>
<LastModified>2025-04-11T12:51:53.013Z</LastModified>
<ETag>"e601cd90d19d045237c6e58345f1a5f8"</ETag>
<Size>577734</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_1ec25a006c3ba2c91d7529dd3bda230b.pkl</Key>
<LastModified>2025-04-11T12:51:10.318Z</LastModified>
<ETag>"bd4f068c08596091030768ae811ae1b5"</ETag>
<Size>590502</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_227f1b52c32f4d8ab119e2811589d337.pkl</Key>
<LastModified>2025-04-11T12:52:32.533Z</LastModified>
<ETag>"c9d90a5253cb3d8809b54adeaccfc2b9"</ETag>
<Size>387331</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_2473f91a928751cb441f49fae2604e48.pkl</Key>
<LastModified>2025-04-11T12:51:27.371Z</LastModified>
<ETag>"e660fec408f78b5d38727ee4ba18edda"</ETag>
<Size>570639</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_2b0b6c6c9287c5c4ef5113cf987a1d9b.pkl</Key>
<LastModified>2025-04-11T12:51:03.640Z</LastModified>
<ETag>"9935e85cf881790ee19aafb7466440cd"</ETag>
<Size>198504</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_3b745a66181ed9d2981ad74a5c832c61.pkl</Key>
<LastModified>2025-04-11T12:53:23.535Z</LastModified>
<ETag>"449f8f1a97d61bbbd547b98f4cf27a48"</ETag>
<Size>272242</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_3e157308dc65450922103fc2ec9cb159.pkl</Key>
<LastModified>2025-04-11T12:53:01.724Z</LastModified>
<ETag>"5d0005bfca18a53e581ce495d67285a9"</ETag>
<Size>130207</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_49f80fa7c6dd53431dd17dcb30e646ae.pkl</Key>
<LastModified>2025-04-11T12:50:36.250Z</LastModified>
<ETag>"00c2a17e1152b51a90403cb7b587dabc"</ETag>
<Size>62606</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_4bf838b090a63a11778cd275bb2e65d3.pkl</Key>
<LastModified>2025-04-11T12:51:44.229Z</LastModified>
<ETag>"5f5b412e187b2c6c22d79f603794f6ca"</ETag>
<Size>780601</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_5293fdc8f807b512b8f514b79dd54f95.pkl</Key>
<LastModified>2025-04-11T12:50:22.795Z</LastModified>
<ETag>"89f7225ab320c5362ee91e761fa82a75"</ETag>
<Size>426661</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_533fcfa2f0fde6df6346b5d24501580a.pkl</Key>
<LastModified>2025-04-11T12:52:51.284Z</LastModified>
<ETag>"65374df78c791bc989d25949e2df5ebd"</ETag>
<Size>78936</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_55f11bef51a72098bd9e1fcaf28e8939.pkl</Key>
<LastModified>2025-04-11T12:50:48.494Z</LastModified>
<ETag>"43f89208ce02cc2f2d3f71c93d330183"</ETag>
<Size>418172</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_582548504db8984a15acce3e49f95575.pkl</Key>
<LastModified>2025-04-11T12:52:08.375Z</LastModified>
<ETag>"21346ba3adaa18dc183a15aaf58abb5f"</ETag>
<Size>786160</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_618bfca119eb0a78e5b88f71d7050618.pkl</Key>
<LastModified>2025-04-11T12:52:43.813Z</LastModified>
<ETag>"a3f0b1f372dfa59b95e7dc464bf47b42"</ETag>
<Size>535404</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_628045f33e690943691e49d01e6dee43.pkl</Key>
<LastModified>2025-04-11T12:52:58.048Z</LastModified>
<ETag>"f0f394c8a4febe467d6bfd41868fe667"</ETag>
<Size>911155</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_632b35643e54bca06e234a9192ba0e79.pkl</Key>
<LastModified>2025-04-11T12:53:17.838Z</LastModified>
<ETag>"25303316fa3eeaa410372509f7f6b6b3"</ETag>
<Size>61144</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_63f9bef57ceb96566040d4fcf61cc68b.pkl</Key>
<LastModified>2025-04-11T12:51:21.690Z</LastModified>
<ETag>"40cdf5316275dae87fd7e4cae7a09eed"</ETag>
<Size>413352</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_677c5bca1c7d6ace1388141bdfb15765.pkl</Key>
<LastModified>2025-04-11T12:51:35.706Z</LastModified>
<ETag>"f722063ceefd8b6a8d5367764e75c448"</ETag>
<Size>125362</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_705ea0772a9b99e73230d9c9741dad00.pkl</Key>
<LastModified>2025-04-11T12:50:39.180Z</LastModified>
<ETag>"9eefcea0e6676683638935b23aca1059"</ETag>
<Size>119278</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_76918a89b8bd994063b1e687c029250c.pkl</Key>
<LastModified>2025-04-11T12:50:15.272Z</LastModified>
<ETag>"95b01cb29165a771abad4de3056a77d6"</ETag>
<Size>282680</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_7d8513c1ad648bbae8e3d458f23ad8da.pkl</Key>
<LastModified>2025-04-11T12:53:25.895Z</LastModified>
<ETag>"5a84a744786147dbb532556ba1917db4"</ETag>
<Size>83328</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_7e1ef0afc9707b6988035c60006378fd.pkl</Key>
<LastModified>2025-04-11T12:51:32.472Z</LastModified>
<ETag>"7eea87ad78d35ddaa04d1a0b82402ca2"</ETag>
<Size>396293</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_87502194b76247425422d3d21be6173c.pkl</Key>
<LastModified>2025-04-11T12:50:06.725Z</LastModified>
<ETag>"eab06374c1b42316e168d3a7a769e198"</ETag>
<Size>746031</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_8ceef4d416fd107ab6c25f46c19ae5da.pkl</Key>
<LastModified>2025-04-11T12:50:11.635Z</LastModified>
<ETag>"f13ca71bd48c6f567cba95a71ef96e88"</ETag>
<Size>401074</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_8da1a95f4bb531016b344a0140f7d4aa.pkl</Key>
<LastModified>2025-04-11T12:51:00.278Z</LastModified>
<ETag>"7b9625a9ee50b239bb892c2c008b33f2"</ETag>
<Size>426370</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_8fcdf6a66b43ebe08ac56b000190e32f.pkl</Key>
<LastModified>2025-04-11T12:53:40.876Z</LastModified>
<ETag>"38150c21d2d044d4071c18d466fab11e"</ETag>
<Size>79339</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_991d6cdf8df4268ff17f99c09939a900.pkl</Key>
<LastModified>2025-04-11T12:53:29.172Z</LastModified>
<ETag>"f769afa5b183a764e27c75121a2d0dca"</ETag>
<Size>162252</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_99206035f1e593711f3b230d6d600ac1.pkl</Key>
<LastModified>2025-04-11T12:52:04.078Z</LastModified>
<ETag>"8f4c03da44dcfc4c313c2806c0e92a77"</ETag>
<Size>576719</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_99525a64369d30a26633ab7f717467df.pkl</Key>
<LastModified>2025-04-11T12:53:05.106Z</LastModified>
<ETag>"7b23129b5274fd24bd9ec07bc99d3af8"</ETag>
<Size>345231</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_a57e8a934d618f6b945b16f338e01634.pkl</Key>
<LastModified>2025-04-11T12:51:39.604Z</LastModified>
<ETag>"16a7a6227329b6db2f818c2f7735ae57"</ETag>
<Size>418172</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_aa5a27e7fa2e0ddc4d6438cf6ce4afc4.pkl</Key>
<LastModified>2025-04-11T12:53:36.478Z</LastModified>
<ETag>"cddf7e9be8bd6b251555eccb0092b56d"</ETag>
<Size>69808</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_abb4268fbaab1f48417b77896ce2d084.pkl</Key>
<LastModified>2025-04-11T12:53:38.589Z</LastModified>
<ETag>"838c602eadee4fb9f4b3ee5fcaccb63b"</ETag>
<Size>43822</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_af87db7c4e2153de48f7a2d1ab4dfeba.pkl</Key>
<LastModified>2025-04-11T12:53:08.944Z</LastModified>
<ETag>"ac77f5fe1c3119d0ea4389e7f58e0847"</ETag>
<Size>211541</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_b29d8bd88f2fbd8fa25597b22f5f5a06.pkl</Key>
<LastModified>2025-04-11T12:50:42.814Z</LastModified>
<ETag>"3f385cce8aec3531f21a2a56d7f0cc0d"</ETag>
<Size>217826</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_b3efe0dd5d26617ba59a43460887cc2e.pkl</Key>
<LastModified>2025-04-11T12:53:14.778Z</LastModified>
<ETag>"f82adecb46ea214457db4201edb036f7"</ETag>
<Size>73224</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_b9fc4ef84607af05053e1f7a1c9b2dee.pkl</Key>
<LastModified>2025-04-11T12:52:25.528Z</LastModified>
<ETag>"5b4f8e2fc46a8837dc69b00eff8376f1"</ETag>
<Size>527087</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_bc5d04b4f2d950f2417a01a7667c612f.pkl</Key>
<LastModified>2025-04-11T12:50:00.583Z</LastModified>
<ETag>"446e537948469c5fc9cf83890ba24f19"</ETag>
<Size>1010659</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_bf4cd4207a2e4b7f7345a93f52af59d7.pkl</Key>
<LastModified>2025-04-11T12:53:34.287Z</LastModified>
<ETag>"1e50752874ea5a699d940ccc5d6afe09"</ETag>
<Size>56926</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_bfa8c61fcee8cd29cbcc5312e445c5ff.pkl</Key>
<LastModified>2025-04-11T12:51:13.423Z</LastModified>
<ETag>"5b63d15e64347fafa4ea3906a484d8a8"</ETag>
<Size>166994</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_c31181ab0610a68a7920f6ed42dca1ec.pkl</Key>
<LastModified>2025-04-11T12:53:20.224Z</LastModified>
<ETag>"d92bfe7b572756393e3c9ea96aeb3623"</ETag>
<Size>75947</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_c374dc82f8a288e7d0c3d87f33aacbf3.pkl</Key>
<LastModified>2025-04-11T12:50:55.152Z</LastModified>
<ETag>"e8f64c80ce85b19d4ff9f8619e77ae32"</ETag>
<Size>564271</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_c3987e3cca4be9568a8318d74100d6cc.pkl</Key>
<LastModified>2025-04-11T12:51:48.372Z</LastModified>
<ETag>"0f8e35ff8f0225333d4a65ddb659e78d"</ETag>
<Size>259381</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_d3b83550bfe0f8e9480b3ba70fd732f2.pkl</Key>
<LastModified>2025-04-11T12:51:17.998Z</LastModified>
<ETag>"f44fb5da0575e8e92a4c1dfc1ae769e0"</ETag>
<Size>521007</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_d3c8d12f8ad9217b4f3df88418fc84ec.pkl</Key>
<LastModified>2025-04-11T12:52:40.342Z</LastModified>
<ETag>"9e1f8bce54129a90933fc454d58bec3b"</ETag>
<Size>772089</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_d6e42542a6883b5d65ce5b7fbccf4b9b.pkl</Key>
<LastModified>2025-04-11T12:51:56.714Z</LastModified>
<ETag>"30c5fd13e383e0bad0654741c6067b94"</ETag>
<Size>468265</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_da7c8e19318dc530ec95d09294d2d0fb.pkl</Key>
<LastModified>2025-04-11T12:52:47.508Z</LastModified>
<ETag>"639b33b3c120cb82c787417d1bd56cad"</ETag>
<Size>255983</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_de6ba296c96edc3d9fa907b7b9d72010.pkl</Key>
<LastModified>2025-04-11T12:52:21.020Z</LastModified>
<ETag>"d14f224a904303bf43e5cea93407b182"</ETag>
<Size>570639</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_e811baf99bd82d8ea7eb6a9e22ec759b.pkl</Key>
<LastModified>2025-04-11T12:52:12.173Z</LastModified>
<ETag>"ef663edc35188fc91d309b5f12b3d6c2"</ETag>
<Size>281183</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_f96e9f31f7dbe8f11fd09548b03ac684.pkl</Key>
<LastModified>2025-04-11T12:53:12.322Z</LastModified>
<ETag>"127830c7365458ce67f62328535941b9"</ETag>
<Size>166725</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/commodities_saved_models/arima_model_fd8740eeef51975fce673af58100e1dc.pkl</Key>
<LastModified>2025-04-11T12:53:31.841Z</LastModified>
<ETag>"4ed554fe305078c7d48d8f14f2e4ca4d"</ETag>
<Size>158101</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_100bdc7c160cbdb4295a7b9fa243e26a.pkl</Key>
<LastModified>2025-04-12T06:57:36.545Z</LastModified>
<ETag>"c03668c38449cf9433d979927d34dc7a"</ETag>
<Size>49600</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_1504c4369bfb4acd29b2e036ed2ec032.pkl</Key>
<LastModified>2025-04-12T06:56:14.471Z</LastModified>
<ETag>"8fad040347323b7201f9370457b6a3d7"</ETag>
<Size>241080</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_211ef89e3e48b5dabc9f10546e711b93.pkl</Key>
<LastModified>2025-04-12T06:57:34.334Z</LastModified>
<ETag>"936fbf71488da07f22a0699d05cce34e"</ETag>
<Size>372997</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_287621d74b02a105769fcb121efef0f6.pkl</Key>
<LastModified>2025-04-12T06:57:17.914Z</LastModified>
<ETag>"825fb299d73e43c4452eb08ab6ccc376"</ETag>
<Size>187605</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_3da490d055119f2d5fa18ff29a9c3ef6.pkl</Key>
<LastModified>2025-04-12T06:56:29.911Z</LastModified>
<ETag>"b038c365cc906e6a9f7ef55ccfbc27fb"</ETag>
<Size>343125</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_3dc2f98f51a08642643cd7e1aaa6f7bf.pkl</Key>
<LastModified>2025-04-12T06:56:38.463Z</LastModified>
<ETag>"6af619d11e103bc824233bdd28e709f9"</ETag>
<Size>539135</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_425188585584291e417574f3b7bc9ade.pkl</Key>
<LastModified>2025-04-12T06:57:28.164Z</LastModified>
<ETag>"a1fe770e00fc2543cce61318cfd1754e"</ETag>
<Size>107775</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_46daf60519d107dba885afbf32a83114.pkl</Key>
<LastModified>2025-04-12T06:57:45.352Z</LastModified>
<ETag>"825f1ba34a840bf5b6f8a3d0b853a6a7"</ETag>
<Size>101240</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_50da7090132fa848416c132f8b05fab8.pkl</Key>
<LastModified>2025-04-12T06:56:33.161Z</LastModified>
<ETag>"a280e00a7dfba8ec0aa79a4d0d333ea0"</ETag>
<Size>166143</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_587ea84f0799705e6c7a0f9a7ca7b636.pkl</Key>
<LastModified>2025-04-12T06:56:21.941Z</LastModified>
<ETag>"91bde017672fda9d390950f9467e740f"</ETag>
<Size>787830</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_7d0556adae3db00e807d8d48a94d171c.pkl</Key>
<LastModified>2025-04-12T06:56:48.144Z</LastModified>
<ETag>"85bbd4a7709444d30b687ddc87727ddc"</ETag>
<Size>864175</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_987b8aabc5c2025da6a2a4538bab1def.pkl</Key>
<LastModified>2025-04-12T06:57:14.855Z</LastModified>
<ETag>"cfe375c99d8fa3a524a48c0d94e46028"</ETag>
<Size>601503</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_9e49da89a4fa1678e9bdb30cefc4b3db.pkl</Key>
<LastModified>2025-04-12T06:57:40.103Z</LastModified>
<ETag>"193a607b9d377a486bb44f6531c380ed"</ETag>
<Size>254207</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_a6a9e7c8eae043a336453f2f7265ce03.pkl</Key>
<LastModified>2025-04-12T06:57:48.267Z</LastModified>
<ETag>"43c7732e764c31c1b47a222164d91fad"</ETag>
<Size>158101</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_af53e0d54a1037dd6713b287013502b8.pkl</Key>
<LastModified>2025-04-12T06:56:26.110Z</LastModified>
<ETag>"418de037670229168044966f2b016c4d"</ETag>
<Size>370578</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_b340a75b315d96d9eab42cda700a56c9.pkl</Key>
<LastModified>2025-04-12T06:57:25.198Z</LastModified>
<ETag>"9c8bdc5e419ac1e1bb6d411452520e7d"</ETag>
<Size>322677</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_b3d3d963f59a4f1ea4824d5af9111340.pkl</Key>
<LastModified>2025-04-12T06:57:09.219Z</LastModified>
<ETag>"a423ccd76c833049d88759efb582de41"</ETag>
<Size>750367</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_ba594c1afe9381cf48037f61b3130a13.pkl</Key>
<LastModified>2025-04-12T06:57:21.394Z</LastModified>
<ETag>"fe3015ebb3b929a845e6b57eaaa37d85"</ETag>
<Size>235445</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_bd14fb01213332e76677dd9973ba9efa.pkl</Key>
<LastModified>2025-04-12T06:57:42.506Z</LastModified>
<ETag>"334d4349f2a6ed35d4fa538ea61e134f"</ETag>
<Size>67307</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_c891febfface83b3153a1d12800975f0.pkl</Key>
<LastModified>2025-04-12T06:56:52.741Z</LastModified>
<ETag>"2fb24d265433f61dda3c99dd3366c9c2"</ETag>
<Size>392005</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_cc66f34e621f8e404ad44b9a978ea2fd.pkl</Key>
<LastModified>2025-04-12T06:57:03.106Z</LastModified>
<ETag>"3c1c948b22b01c9094b755bb03e8ba03"</ETag>
<Size>545823</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_df33c820fdd3bd9f139d948554d1e813.pkl</Key>
<LastModified>2025-04-12T06:56:57.960Z</LastModified>
<ETag>"edc001813db9c40697428bebffb57466"</ETag>
<Size>610991</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_e5e4f6100a26c314e26a8cc71e15096d.pkl</Key>
<LastModified>2025-04-12T06:56:41.153Z</LastModified>
<ETag>"a27ce5a310b001377a27b68c57ee4d4e"</ETag>
<Size>126107</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/fruits_saved_models/arima_model_e9eb4c7cf9402b51e44b98141f351c8b.pkl</Key>
<LastModified>2025-04-12T06:57:30.506Z</LastModified>
<ETag>"942b0d291fd4af6c311016623083f4d8"</ETag>
<Size>49870</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/</Key>
<LastModified>2025-04-09T09:32:30.403Z</LastModified>
<ETag>"1441a7909c087dbbe7ce59881b9df8b9"</ETag>
<Size>15</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_02aae4780e1275be739244b10eb070c2.pkl</Key>
<LastModified>2025-04-11T12:48:22.029Z</LastModified>
<ETag>"3c193ae06afa7bbf2c605bb589566836"</ETag>
<Size>1286691</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_0c19e477fb27c285a611e8c850ad0abe.pkl</Key>
<LastModified>2025-04-11T12:48:08.239Z</LastModified>
<ETag>"2727a3378e4c26ca83f3fe189ed25c3c"</ETag>
<Size>355458</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_17fdb67e711f687d266f9f004bc9162c.pkl</Key>
<LastModified>2025-04-11T12:48:39.943Z</LastModified>
<ETag>"99148eb1e42d4dc4260c8be827e595ee"</ETag>
<Size>833923</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_227f1b52c32f4d8ab119e2811589d337.pkl</Key>
<LastModified>2025-04-11T12:46:57.391Z</LastModified>
<ETag>"3a4f82fe8d170338b3213ff6965c0277"</ETag>
<Size>1286691</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_2296fe872960d5ae15f3f87317a0350c.pkl</Key>
<LastModified>2025-04-11T12:46:33.528Z</LastModified>
<ETag>"112f26ce8396e226288e83aa68f5f181"</ETag>
<Size>1286691</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_2b0b6c6c9287c5c4ef5113cf987a1d9b.pkl</Key>
<LastModified>2025-04-11T12:46:47.920Z</LastModified>
<ETag>"9e0ca19813a1195cd717162d2f646b54"</ETag>
<Size>444338</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_2b294dd2ba094e95245802972b8139ef.pkl</Key>
<LastModified>2025-04-11T12:47:34.263Z</LastModified>
<ETag>"73de8ad9b30b0207ccef350a11ac9a8e"</ETag>
<Size>651135</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_2e00d072aa0ff521f7a40cf17de0a7c7.pkl</Key>
<LastModified>2025-04-11T12:47:52.997Z</LastModified>
<ETag>"dad1e29a47aa7804a7cb8e22a5f9b015"</ETag>
<Size>737919</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_2ed6f4c0d53ea322a6e2a50c1200b7f6.pkl</Key>
<LastModified>2025-04-11T12:48:00.992Z</LastModified>
<ETag>"55a73f991ce9107a43afa539c1607436"</ETag>
<Size>1242515</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_4312bb6d0674a989acff2d98cdd9f4ac.pkl</Key>
<LastModified>2025-04-11T12:47:25.544Z</LastModified>
<ETag>"e16847c2155799981ff1ed1c5c975cb1"</ETag>
<Size>526213</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_49b76ecfbe861dd1c15d27b53830e574.pkl</Key>
<LastModified>2025-04-11T12:48:25.663Z</LastModified>
<ETag>"bc8009d54addc8c14d1fe26cb3d7056b"</ETag>
<Size>713103</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_53a8c445bbc876962de0afa269bc4b91.pkl</Key>
<LastModified>2025-04-11T12:47:41.006Z</LastModified>
<ETag>"ecbd97b6ece2e4a9f04bb2369df85f25"</ETag>
<Size>183925</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_573bd610a5184f3d842198531db4be86.pkl</Key>
<LastModified>2025-04-11T12:47:07.022Z</LastModified>
<ETag>"81babff70f3798d4656c049fa977a9c1"</ETag>
<Size>1286691</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_57d0c9c23c553dd11d3179bab7759753.pkl</Key>
<LastModified>2025-04-11T12:46:44.735Z</LastModified>
<ETag>"2a9ce6e7bb7c3c8d1de64692d1a9d5e3"</ETag>
<Size>898745</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_582548504db8984a15acce3e49f95575.pkl</Key>
<LastModified>2025-04-11T12:46:52.289Z</LastModified>
<ETag>"a2ea0c541d3411a1e48712b11788ddbe"</ETag>
<Size>991481</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_5d65fd1db28500a688534bbabf9eacb1.pkl</Key>
<LastModified>2025-04-11T12:45:44.288Z</LastModified>
<ETag>"94fb354fe9b531cb3a130ab2feeb0294"</ETag>
<Size>991481</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_61aea3c4b4d395d55bb0356c6be6dbaf.pkl</Key>
<LastModified>2025-04-11T12:46:01.776Z</LastModified>
<ETag>"63eb5d55224eb42dc5c7546eed914a39"</ETag>
<Size>1286691</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_6b3ba9b2b3bcd7fa9802edc58eec99e8.pkl</Key>
<LastModified>2025-04-11T12:47:44.093Z</LastModified>
<ETag>"74daccdc3e4128f0d4d28b4d02143365"</ETag>
<Size>361013</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_6c06936af6d9e9177defa74176b2a3bb.pkl</Key>
<LastModified>2025-04-11T12:48:57.403Z</LastModified>
<ETag>"d3789107b45f3cdb1c575f2bada1f60c"</ETag>
<Size>278988</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_6fca0cf0668f416391ea400438efecad.pkl</Key>
<LastModified>2025-04-11T12:46:20.894Z</LastModified>
<ETag>"6bf65f4247dc65d7982dfd41bd0d9b0b"</ETag>
<Size>526213</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_6ffb48d52955d838d0232ae155d842e6.pkl</Key>
<LastModified>2025-04-11T12:47:48.802Z</LastModified>
<ETag>"a40e433cee6d987719e09c0b29f96922"</ETag>
<Size>1286691</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_7ca44738acb382b89e02435bd7c22424.pkl</Key>
<LastModified>2025-04-11T12:46:08.641Z</LastModified>
<ETag>"69f61363590e70f6a3420a8f182ead69"</ETag>
<Size>755369</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_7e0ca41c93fb69a53e16a6d4af1e5b6c.pkl</Key>
<LastModified>2025-04-11T12:48:51.977Z</LastModified>
<ETag>"4e0f0ec3ce1042c0b06bedf8de244221"</ETag>
<Size>239788</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_814c79abdb2371a31615e8c24a02a701.pkl</Key>
<LastModified>2025-04-11T12:47:38.031Z</LastModified>
<ETag>"c16a9e4f8643191d1e4461962756f888"</ETag>
<Size>241397</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_8bb5f70e8e15db269ca7b671ad9aa8d9.pkl</Key>
<LastModified>2025-04-11T12:47:21.267Z</LastModified>
<ETag>"953698fdde7b7c59256abff0d39a8885"</ETag>
<Size>729961</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_901249aa9278f78e0417357b304a7583.pkl</Key>
<LastModified>2025-04-11T12:47:16.431Z</LastModified>
<ETag>"6dfd36a89e7e8b7d1551d43254370de2"</ETag>
<Size>737919</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_952a71343bca62e6737a30576a86347b.pkl</Key>
<LastModified>2025-04-11T12:45:51.961Z</LastModified>
<ETag>"a359627ccfff6b199e34a491582230ed"</ETag>
<Size>755369</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_9bd5e81b912ad03e33db79932768ad79.pkl</Key>
<LastModified>2025-04-11T12:47:30.462Z</LastModified>
<ETag>"b2e628ed4d8de56377d0b25a62cda7c4"</ETag>
<Size>814902</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_a508f0e1c78b869b9625a84190a881c8.pkl</Key>
<LastModified>2025-04-11T12:48:48.266Z</LastModified>
<ETag>"3d43d675c0fc364f006644ed1331ca4c"</ETag>
<Size>68536</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_af86bc32c629e75ce0910a03b545f51a.pkl</Key>
<LastModified>2025-04-11T12:47:56.683Z</LastModified>
<ETag>"ab3826157906d38cf7e1616ae1eb8441"</ETag>
<Size>526213</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_b14c8f5cbd856d7fd800c8c3ab40ad69.pkl</Key>
<LastModified>2025-04-11T12:48:04.882Z</LastModified>
<ETag>"904d05dcdaee8be9c94f51f7ed5da666"</ETag>
<Size>243048</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_beb2aec6648c6ca021db827a61145547.pkl</Key>
<LastModified>2025-04-11T12:46:39.584Z</LastModified>
<ETag>"9f1226c9dff43d64a201dfdb73aaba2a"</ETag>
<Size>737919</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_c2923048f644a7bb617ec41f0ea79370.pkl</Key>
<LastModified>2025-04-11T12:48:12.455Z</LastModified>
<ETag>"f2007b693136453c5c2307dc1137f282"</ETag>
<Size>737919</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_c30cb0225dab7a0a11b08cc7d4830a4d.pkl</Key>
<LastModified>2025-04-11T12:47:02.511Z</LastModified>
<ETag>"1bd5dc060f7cdf820999ce64c93cf5a8"</ETag>
<Size>898745</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_d27c18035d51fe210333208de2b7c0c9.pkl</Key>
<LastModified>2025-04-11T12:48:34.376Z</LastModified>
<ETag>"11c2c556043f03ca834d23e55d8051e9"</ETag>
<Size>1286691</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_d3b83550bfe0f8e9480b3ba70fd732f2.pkl</Key>
<LastModified>2025-04-11T12:45:36.384Z</LastModified>
<ETag>"74dcbadd72728eafa37f3127dfc8dec8"</ETag>
<Size>991481</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_d77da01ab7065105c3d353804f60f1f5.pkl</Key>
<LastModified>2025-04-11T12:47:09.672Z</LastModified>
<ETag>"cc64dc0e9e68bf99705859469288b9c6"</ETag>
<Size>172853</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_d83dfe86a61903671a11b8bca35e78e1.pkl</Key>
<LastModified>2025-04-11T12:47:12.443Z</LastModified>
<ETag>"ffd68a38d19bb9ff9add610c47a7f87b"</ETag>
<Size>159669</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_d913c68c220da9980eebb79c73cc78a2.pkl</Key>
<LastModified>2025-04-11T12:48:45.929Z</LastModified>
<ETag>"d5f6f0b64b61439e7b54e6905fb35f15"</ETag>
<Size>53688</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_de6e247e91ef058eefa4c1e4576ee6e0.pkl</Key>
<LastModified>2025-04-11T12:48:43.581Z</LastModified>
<ETag>"9f47db6f5f47d03fb54c780584c4a494"</ETag>
<Size>283436</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_e33ad8c9adb63463456dfacdb7157c3c.pkl</Key>
<LastModified>2025-04-11T12:46:25.758Z</LastModified>
<ETag>"b51ad8e9d3a21d1db18080c9631fe0cc"</ETag>
<Size>737919</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_e7f9e460d7f49d681ba5054e76a0e5ae.pkl</Key>
<LastModified>2025-04-11T12:48:54.368Z</LastModified>
<ETag>"f85f74c2dc61eb8f35ce08aee2ce1c00"</ETag>
<Size>69995</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_e911e5976540994f94b4579599ed80bd.pkl</Key>
<LastModified>2025-04-11T12:48:17.472Z</LastModified>
<ETag>"93233f81ee01560140d763a0a972e9b3"</ETag>
<Size>586361</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_ea48f7e71ae1c5d9de5a6b5a5eb47f49.pkl</Key>
<LastModified>2025-04-11T12:48:29.038Z</LastModified>
<ETag>"1fa71cfd9acdc748bccc871ecb5f7688"</ETag>
<Size>157166</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_ee899c66e43ecca7bf84565bd39ee8b0.pkl</Key>
<LastModified>2025-04-11T12:45:26.041Z</LastModified>
<ETag>"d0d42967e440a084a26a6a3d1fbb82f8"</ETag>
<Size>1286691</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Contents>
<Key>apmc-ml-models/ml_models/vegetables_saved_models/arima_model_f18e5623f0f9034e8dfe8978635608eb.pkl</Key>
<LastModified>2025-04-11T12:46:17.551Z</LastModified>
<ETag>"309c9c07ad272c822274d0211de6eb30"</ETag>
<Size>1286691</Size>
<StorageClass>STANDARD</StorageClass>
<Owner>
<ID>18829799</ID>
<DisplayName>18829799</DisplayName>
</Owner>
<Type>Normal</Type>
</Contents>
<Marker/>
</ListBucketResult>